In [174]:
import torch
import numpy as np

In [175]:
w = torch.tensor([2.0], requires_grad=True)
b = torch.tensor([1.0], requires_grad=True)

# Input
x = torch.tensor([3.0])

In [176]:
# Forward pass
y_pred = w * x + b
print("y_pred:", {y_pred.item()})

y_pred: {7.0}


In [177]:
# Loss
y_true = torch.tensor([10.0])
loss = (y_pred - y_true) ** 2
print("Loss:", {loss.item()})

Loss: {9.0}


In [178]:
print("Avant backward: w.grad =", w.grad)

Avant backward: w.grad = None


In [179]:
loss.backward()

In [180]:
print("Après backward: w.grad =", w.grad)
print("Apres backward: b.grad =", b.grad)

Après backward: w.grad = tensor([-18.])
Apres backward: b.grad = tensor([-6.])


In [181]:
learning_rate = 0.01
with torch.no_grad(): # Désactiver le tracking des gradients pour l'update w = w + alpha * w
    w -= learning_rate * w.grad
    b -= learning_rate * b.grad

In [182]:
print("Nouveau poids : w =", w, "b =", b)

Nouveau poids : w = tensor([2.1800], requires_grad=True) b = tensor([1.0600], requires_grad=True)


In [183]:
# Réinitialiser les gradients
w.grad.zero_()
b.grad.zero_()
print("Après zero_(): w.grad =", w.grad, "b.grad =", b.grad)

Après zero_(): w.grad = tensor([0.]) b.grad = tensor([0.])


In [184]:
import torch.nn as nn

perceptron = nn.Linear(1, 1)

# Voir les poids initiaux
print("Poids :", perceptron.weight)
print("Biais :", perceptron.bias)

Poids : Parameter containing:
tensor([[0.7008]], requires_grad=True)
Biais : Parameter containing:
tensor([0.7492], requires_grad=True)


In [185]:
# Forward
x = torch.tensor([3.0]) # Shape (batch_size, features)
print(f"input : {x}")
y_pred = perceptron(x)
print(y_pred)


input : tensor([3.])
tensor([2.8517], grad_fn=<ViewBackward0>)


In [186]:
# Loss
y_true = torch.tensor([10.0])
criterion = nn.MSELoss()
loss = criterion(y_pred, y_true)

In [187]:
loss

tensor(51.0984, grad_fn=<MseLossBackward0>)

In [188]:
# Backward
loss.backward()

In [189]:
print(f"Gradient de poids : {perceptron.weight.grad}")
print(f"Gradient de biais : {perceptron.bias.grad}")

Gradient de poids : tensor([[-42.8899]])
Gradient de biais : tensor([-14.2966])


In [190]:
import torch.optim as optim
optimizer = optim.SGD(perceptron.parameters(), lr=0.01)

In [191]:
optimizer.step()

In [192]:
print("Nouveau poids :", perceptron.weight)
print("Nouveau biais :", perceptron.bias)

Nouveau poids : Parameter containing:
tensor([[1.1297]], requires_grad=True)
Nouveau biais : Parameter containing:
tensor([0.8922], requires_grad=True)


In [193]:
optimizer.zero_grad()

In [194]:
print(f"Poids après zero_grad : {perceptron.weight.grad}")
print(f"Biais après zero_grad : {perceptron.bias.grad}")

Poids après zero_grad : None
Biais après zero_grad : None


In [195]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 4)
        self.fc2 = nn.Linear(4, 3)
        self.fc3 = nn.Linear(3, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [196]:
model = SimpleNet()

In [197]:
model

SimpleNet(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
  (fc3): Linear(in_features=3, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [198]:
x = torch.tensor([1.0, 2.0])
y = torch.tensor([5.0])
y_pred = model(x)
loss = nn.MSELoss()(y_pred, y)

loss.backward()

In [199]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Compose, Normalize
import onnxruntime as rt
import matplotlib.pyplot as plt


In [200]:
training_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor()
)



In [201]:
training_dataloader = DataLoader(training_data, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
test_data = DataLoader(test_data, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [202]:
print(training_data.classes)
print(training_data.data.size())

['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']
torch.Size([60000, 28, 28])


In [203]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(64*3*3, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [204]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
model = CNN().to(device)
print(model)

CNN(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=576, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [205]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [206]:
x = torch.rand(1, 1, 28, 28, device=device)
logits = model(x)

In [207]:
logits

tensor([[ 0.0574, -0.0192,  0.0821, -0.0984,  0.0027, -0.1120,  0.1007, -0.0446,
         -0.0677,  0.0264]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [208]:
pred_probab = nn.Softmax(dim=1)(logits)
pred_probab

tensor([[0.1064, 0.0986, 0.1091, 0.0911, 0.1007, 0.0898, 0.1111, 0.0961, 0.0939,
         0.1032]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [209]:
y_pred = pred_probab.argmax(dim=1)
y_pred

tensor([6], device='cuda:0')

In [210]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch_idx, batch_value in enumerate(dataloader):
        X, y = batch_value
        X, y = X.to(device), y.to(device)

        # Forward pass
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch_idx % 100 == 0:
            loss, current = loss.item(), (batch_idx+1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5}]")


In [211]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
            

In [212]:
epochs = 10
for t in range(epochs):
    running_loss = 0.0
    model.train()
    print(f"Epoch {t+1}\n-------------------------------")
    for i, data in enumerate(training_dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f'[Epoch {t + 1}, Batch {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0
        
    # train(training_dataloader, model, loss_fn, optimizer)
    test(test_data, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
[Epoch 1, Batch   100] loss: 2.304
[Epoch 1, Batch   200] loss: 2.302
[Epoch 1, Batch   300] loss: 2.300
[Epoch 1, Batch   400] loss: 2.300
[Epoch 1, Batch   500] loss: 2.299
[Epoch 1, Batch   600] loss: 2.297
[Epoch 1, Batch   700] loss: 2.296
[Epoch 1, Batch   800] loss: 2.294
[Epoch 1, Batch   900] loss: 2.291
Test Error: 
 Accuracy: 31.6%, Avg loss: 2.285972 

Epoch 2
-------------------------------
[Epoch 2, Batch   100] loss: 2.286
[Epoch 2, Batch   200] loss: 2.280
[Epoch 2, Batch   300] loss: 2.270
[Epoch 2, Batch   400] loss: 2.256
[Epoch 2, Batch   500] loss: 2.223
[Epoch 2, Batch   600] loss: 2.148
[Epoch 2, Batch   700] loss: 1.974
[Epoch 2, Batch   800] loss: 1.662
[Epoch 2, Batch   900] loss: 1.355
Test Error: 
 Accuracy: 72.0%, Avg loss: 0.917221 

Epoch 3
-------------------------------
[Epoch 3, Batch   100] loss: 1.067
[Epoch 3, Batch   200] loss: 0.914
[Epoch 3, Batch   300] loss: 0.834
[Epoch 3, Batch   400] loss: 0.751
[Epoch

In [213]:
example_input = (torch.randn(1, 1, 28, 28),)
model.to("cpu")
model.eval()
onnx_program = torch.onnx.export(model, example_input, input_names=["input"], output_names=["output"], dynamo=True)

[torch.onnx] Obtain model graph for `CNN([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `CNN([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


In [214]:
onnx_program.save("model.onnx")
sess = rt.InferenceSession("model.onnx", providers= rt.get_available_providers())
input_name = sess.get_inputs()[0].name
pred_onnx = sess.run(None, {input_name: example_input[0].numpy()})
print(pred_onnx)

[array([[-4.4135857 , -1.0919702 ,  1.0299526 ,  6.5817504 , -5.1239715 ,
         5.4989266 , -3.1605928 , -0.17881511,  3.6822152 , -0.38163128]],
      dtype=float32)]
